In [ ]:
import pypsa
import logging
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

plt.style.use("bmh")
sns.set_palette("colorblind")

logging.getLogger("gurobipy").setLevel(logging.CRITICAL)

In [ ]:
url = "https://tubcloud.tu-berlin.de/s/kpWaraGc9LeaxLK/download/network-cem.nc"

n = pypsa.Network(url)

n.remove("GlobalConstraint", "CO2Limit")
n.remove("Load", "demand")
n.remove("StorageUnit", "battery storage")

n.mremove("Carrier", n.carriers.index)
n.mremove("Generator", ["OCGT", "offwind"])

n.generators.marginal_cost = 0

## One Segment

In [ ]:
ELASTIC_INTERCEPT = 2000
LOAD = 100

In [ ]:
n_notrick = n.copy()

n_notrick.add(
    "Generator",
    "load",
    bus="Germany",
    carrier="load",
    marginal_cost=ELASTIC_INTERCEPT,
    marginal_cost_quadratic=ELASTIC_INTERCEPT / (2 * LOAD),
    p_max_pu=0,
    p_min_pu=-1,
    p_nom=LOAD,
)

In [ ]:
n_trick = n.copy()

n_trick.add(
    "Generator",
    "load-shedding",
    bus="Germany",
    carrier="load",
    p_nom=LOAD,
    marginal_cost_quadratic=ELASTIC_INTERCEPT / (2 * LOAD),
)

n_trick.add(
    "Load",
    "load",
    bus="Germany",
    carrier="load",
    p_set=LOAD,
)

In [ ]:
n_trick.optimize(solver_name="gurobi", solver_options={"BarConvTol": 1e-9})

In [ ]:
n_notrick.optimize(solver_name="gurobi", solver_options={"BarConvTol": 1e-9})

In [ ]:
n_trick.statistics().round() - n_notrick.statistics().round()

In [ ]:
(
    n_notrick.buses_t.marginal_price["Germany"]
    - n_trick.buses_t.marginal_price["Germany"]
).plot(figsize=(10, 2))

## Two Segments

In [ ]:
ELASTIC_INTERCEPT_1 = 6000
ELASTIC_SLOPE_1 = 60
ELASTIC_LOAD_1 = 95

ELASTIC_INTERCEPT_2 = 300
ELASTIC_SLOPE_2 = 30
ELASTIC_LOAD_2 = 10

In [ ]:
n_notrick = n.copy()

n_notrick.add(
    "Generator",
    "load-segment-1",
    bus="Germany",
    carrier="load",
    marginal_cost=ELASTIC_INTERCEPT_1,
    marginal_cost_quadratic=ELASTIC_SLOPE_1 / 2,
    p_max_pu=0,
    p_min_pu=-1,
    p_nom=ELASTIC_LOAD_1,
)

n_notrick.add(
    "Generator",
    "load-segment-2",
    bus="Germany",
    carrier="load",
    marginal_cost=ELASTIC_INTERCEPT_2,
    marginal_cost_quadratic=ELASTIC_SLOPE_2 / 2,
    p_max_pu=0,
    p_min_pu=-1,
    p_nom=ELASTIC_LOAD_2,
)

In [ ]:
n_trick = n.copy()

n_trick.add(
    "Generator",
    "load-shedding-segment-1",
    bus="Germany",
    carrier="load",
    p_nom=ELASTIC_LOAD_1,
    marginal_cost=ELASTIC_INTERCEPT_1 - ELASTIC_SLOPE_1 * ELASTIC_LOAD_1,
    marginal_cost_quadratic=ELASTIC_SLOPE_1 / 2,
)

n_trick.add(
    "Generator",
    "load-shedding-segment-2",
    bus="Germany",
    carrier="load",
    p_nom=ELASTIC_LOAD_2,
    marginal_cost_quadratic=ELASTIC_SLOPE_2 / 2,
)

n_trick.add(
    "Load",
    "load",
    bus="Germany",
    carrier="load",
    p_set=ELASTIC_LOAD_1 + ELASTIC_LOAD_2,
)

In [ ]:
ELASTIC_INTERCEPT_1 - ELASTIC_SLOPE_1 * ELASTIC_LOAD_1

In [ ]:
n_trick.optimize(solver_name="gurobi")

In [ ]:
n_notrick.optimize(solver_name="gurobi")

In [ ]:
n_trick.statistics().round() - n_notrick.statistics().round()

In [ ]:
(
    n_notrick.buses_t.marginal_price["Germany"]
    - n_trick.buses_t.marginal_price["Germany"]
).plot()

In [ ]:
n_trick.generators_t.p.loc["2015-10", "load-shedding-segment-1"].plot(
    figsize=(20, 3), linewidth=1
)
n_trick.generators_t.p.loc["2015-10", "load-shedding-segment-2"].plot(
    figsize=(20, 3), linewidth=1
)